In [1]:
# used to widen the cells 
from IPython.core.display import display, HTML
display(HTML("<style>.container { margin-left: 2.5% !important; width:95%; }</style>"))


In [2]:
from algosdk.v2client import algod
from algosdk import mnemonic
from algosdk import transaction

algod_address = "https://testnet-algorand.api.purestake.io/ps2"
algod_token = "fFG6R4JTFq9Hh8RjZy130aMRdUhmOYp68hi9nm4L"
headers = {
   "X-API-Key": algod_token,
}

algod_client = algod.AlgodClient(algod_token, algod_address, headers);


In [4]:
funded_mnemonics= ["rookie similar wire owner wash once fish mosquito glad coffee family venture adult funny melt gas inspire tuna buzz sell dignity pottery gold able bracket",
                  "narrow mimic suffer top suspect follow menu broccoli try snake meat erase napkin lucky client forget sense bread glad eight uniform bacon code about crack", 
                  "city truth device clog grocery sea safe slide glove borrow swap capable trash shaft vast start space distance calm wire hub crush dose ability army", 
                  "unknown arctic antenna country credit december ill practice lawsuit today athlete rescue exit swarm fitness strong minimum soldier wide coffee vacuum piece coil absorb unable", 
                  "gravity adult destroy demand margin coast culture base adjust east banana certain happy daughter bless fiscal fiscal eye forget inspire brain banner evil able bonus"]

funded_accounts = [{'sk': mnemonic.to_private_key(memo), 'pk': mnemonic.to_public_key(memo)} for memo in funded_mnemonics]

In [7]:
def compile(approval_filename='approval.teal', clear_state_filename='clear_state.teal'):

    # pyteal to teal 
    with open(approval_filename, 'w') as f:
        compiled = compileTeal(approval_program(), Mode.Application)
        f.write(compiled)

    with open(clear_state_filename, 'w') as f:
        compiled = compileTeal(clear_state_program(), Mode.Application)
        f.write(compiled)

    # teal to bytecode 
    stdout, stderr = execute(["goal", "clerk", "compile", "-o", approval_filename+'.tok', approval_filename])
    if stderr != "":
        print(stderr)
        raise
    elif len(stdout) < 59:
        print("error in compile teal")
        raise

    stdout, stderr = execute(["goal", "clerk", "compile", "-o", clear_state_filename+'.tok', clear_state_filename])
    if stderr != "":
        print(stderr)
        raise
    elif len(stdout) < 59:
        print("error in compile teal")
        raise

    with open(approval_filename+'.tok', 'rb') as f:
        approval_bytes = f.read()

    with open(clear_state_filename+'.tok', 'rb') as f: 
        clear_state_bytes = f.read() 
        
    return approval_bytes, clear_state_bytes

## options in contract storage

In [6]:
from pyteal import *    
    
def approval_program():     
    on_creation = Seq([    
        App.globalPut(Bytes("Creator"), Txn.sender()),    
        App.globalPut(Bytes("Name"), Txn.application_args[0]),
        Return(Int(1))
    ])    
    
    on_closeout = Return(Int(1))
    on_optin = Return(Int(1))
    
    myStatus = App.globalGetEx(Int(0), Bytes("status"))
    Seq([
        myStatus,
        If(myStatus.hasValue(), myStatus.value(), Bytes("none"))
    ])
    
    
    option_name = Concat(Bytes("option_"), Txn.application_args[1])
    
    option_tally = App.globalGetEx(Int(0), option_name)
    on_vote = Seq([
        option_tally, 
        If(option_tally.hasValue(),
           Seq([App.globalPut(option_name, option_tally.value()+Int(1)), Return(Int(1))]),
           Return(Int(0))
          )
    ])
    
    on_add_option = Seq([
        App.globalPut(option_name, 0),
        Return(Int(1))
    ])
        
    program = Cond(    
            [Txn.application_id() == Int(0), on_creation],    
            [Txn.on_completion() == OnComplete.DeleteApplication, Return(Int(0))], 
            [Txn.on_completion() == OnComplete.UpdateApplication, Return(Int(0))],    
            [Txn.on_completion() == OnComplete.OptIn, on_optin],
            [Txn.application_args[0] == Bytes("vote"), on_vote],
            [Txn.application_args[0] == Bytes("add_opiton"), on_add_option]
    )    
    return program    
    
    
def clear_state_program():     
    program = Return(Int(1))
    return program    
    


In [8]:
from algosdk.future import transaction
from algosdk.future.transaction import StateSchema, OnComplete